## Polynomial Chaos Expansion Example 5

Author: Katiana Kontolati \
Date: December 8, 2020

In this example, PCE is used to generate a surrogate model for a given set of 5D data.

### Friedman function

$$ f(x) = 10 sin(\pi x_1x_2) + 20(x_3 - 0.5)^2 + 10x_4 + 5x_5$$

**Description:**  Dimensions: 5

**Input Domain:**  This function is evaluated on the hypercube $x_i \in [0,1]$ for all i = 1, …, 5.

**Reference:**  Friedman, J. H. (1991). Multivariate adaptive regression splines. The annals of statistics, 19(1), 1-67.

Import necessary libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from UQpy.Surrogates import *

Define the function.

In [2]:
def function(x):
    return 10*np.sin(np.pi*x[:,0]*x[:,1]) + 20*(x[:,2]-0.5)**2 + 10*x[:,3] + 5*x[:,4]

Create a distribution object, generate samples and evaluate the function at the samples.

In [3]:
np.random.seed(1)

dist = Uniform(loc=0, scale=1)  

marg = [dist]*5
joint = JointInd(marginals=marg)

n_samples = 200
x = joint.rvs(n_samples)
y = function(x)  

Create an object from the PCE class.

In [4]:
max_degree = 3
polys = Polynomials(dist_object=joint, degree=max_degree) 

Compute PCE coefficients using least squares regression.

In [5]:
lstsq = PolyChaosLstsq(poly_object=polys)
pce = PCE(method=lstsq) 
pce.fit(x,y) 

Compute PCE coefficients using Lasso regression.

In [6]:
lasso = PolyChaosLasso(poly_object=polys, learning_rate=0.05, iterations=1000, penalty=0.5)
pce2 = PCE(method=lasso) 
pce2.fit(x,y) 

Compute PCE coefficients using Ridge regression.

In [7]:
ridge = PolyChaosRidge(poly_object=polys, learning_rate=0.01, iterations=1000, penalty=0.5)
pce3 = PCE(method=ridge) 
pce3.fit(x,y)

### Error Estimation

Validation error.

In [8]:
n_samples = 100
x_val = joint.rvs(n_samples)
y_val = function(x_val)

error = ErrorEstimation(surr_object=pce)
error2 = ErrorEstimation(surr_object=pce2)
error3 = ErrorEstimation(surr_object=pce3)
print('Error from least squares regression is: ', error.validation(x_val, y_val))
print('Error from LASSO is: ', error2.validation(x_val, y_val))
print('Error from Ridge is: ', error3.validation(x_val, y_val))

Error from least squares regression is:  0.0035946
Error from LASSO is:  0.0036024
Error from Ridge is:  0.0036999


### Moment Estimation

Returns mean and variance of the PCE surrogate.

In [9]:
n_mc = 1000000
x_mc = joint.rvs(n_mc)
y_mc = function(x_mc)    
mu = np.mean(y_mc)

print('Moments from least squares regression :', MomentEstimation(surr_object=pce).get())
print('Moments from LASSO regression :', MomentEstimation(surr_object=pce2).get())
print('Moments from Ridge regression :', MomentEstimation(surr_object=pce3).get())
print('Moments from MC integration: ', (round((1/n_mc)*np.sum(y_mc),6), round((1/n_mc)*np.sum((y_mc-mu)**2),6)))

Moments from least squares regression : (14.386271, 23.806292)
Moments from LASSO regression : (14.384072, 23.770833)
Moments from Ridge regression : (14.375257, 23.572673)
Moments from MC integration:  (14.414001, 23.852139)
